# Week 12: Capstone Project Part 5.3

- Done by: A Alkaff Ahamed
- Grade: Pending
- 2 July 2025


## Learning Outcome Addressed
- Apply techniques for effective communication and coordination among multiple agents
- Leverage LLMs for task planning, execution and autonomous problem-solving
- Assess the limitations, challenges and emerging trends in multi-agent AI systems


**Exercise: Multi-service Agent System** 

## Objective 

Build a recommendation system that combines weather data and local events to suggest activities, demonstrating practical multi-agent coordination. 

Prerequisites 

- OpenAI API key 
- WeatherAPI.com free account 
- Python 3.8+ 
- SQLite3 
- Previous exercises completed 

**Setup** 

```python
import sqlite3 
 
def setup_database(): 
    conn = sqlite3.connect('events.db') 
    c = conn.cursor() 
     
    c.execute(''' 
        CREATE TABLE IF NOT EXISTS events ( 
            id INTEGER PRIMARY KEY, 
            name TEXT, 
            type TEXT,  -- 'indoor' or 'outdoor' 
            description TEXT, 
            location TEXT, 
            date TEXT 
        ) 
    ''') 
     
    # Sample events 
    events = [ 
        ('Summer Concert', 'outdoor', 'Live music in the park', 'Central Park', '2025-07-15'), 
        ('Art Exhibition', 'indoor', 'Modern art showcase', 'City Gallery', '2025-07-15'), 
        ('Food Festival', 'outdoor', 'International cuisine', 'Waterfront', '2025-07-16'), 
        ('Theater Show', 'indoor', 'Classical drama', 'Grand Theater', '2025-07-16') 
    ] 
     
    c.executemany('INSERT OR IGNORE INTO events (name, type, description, location, date) VALUES (?,?,?,?,?)', events) 
    conn.commit() 
    conn.close() 
 
if __name__ == "__main__": 
    setup_database() 
```

1. Create `config.py`: 

```
OPENAI_API_KEY = "your-key-here" 
WEATHER_API_KEY = "your-weather-api-key" 
```

**Implementation** 

Create `recommendation_system.py`: 

```python
import openai 
import requests 
import sqlite3 
from datetime import datetime 
from config import OPENAI_API_KEY, WEATHER_API_KEY 
 
class WeatherAgent: 
    def __init__(self, api_key): 
        self.api_key = api_key 
     
    def get_weather(self, location, date): 
        # Use current weather since we're dealing with same-day recommendations 
        url = f"http://api.weatherapi.com/v1/current.json" 
        params = { 
            "key": self.api_key, 
            "q": location, 
            "aqi": "no" 
        } 
        try: 
            response = requests.get(url, params=params) 
            response.raise_for_status() 
            return response.json() 
        except requests.exceptions.RequestException as e: 
            raise Exception(f"Weather API error: {str(e)}") 
 
class EventAgent: 
    def get_events(self, date, event_type=None): 
        conn = sqlite3.connect('events.db') 
        c = conn.cursor() 
         
        try: 
            if event_type: 
                c.execute('SELECT * FROM events WHERE date = ? AND type = ?', (date, event_type)) 
            else: 
                c.execute('SELECT * FROM events WHERE date = ?', (date,)) 
                 
            events = c.fetchall() 
            return events 
        except sqlite3.Error as e: 
            raise Exception(f"Database error: {str(e)}") 
        finally: 
            conn.close() 
 
class RecommendationAgent: 
    def __init__(self, openai_api_key): 
        self.api_key = openai_api_key 
        openai.api_key = openai_api_key 
     
    def generate_recommendation(self, weather_data, events): 
        # Create context for GPT 
        try: 
            # Handle both current and forecast data 
            if 'current' in weather_data: 
                weather_condition = weather_data['current']['condition']['text'] 
                temperature = weather_data['current']['temp_c'] 
                context = f"Weather: {weather_condition}, Temperature: {temperature}°C\n\n" 
            else: 
                context = "Weather data unavailable\n\n" 
             
            context += "Available events:\n" 
            for event in events: 
                context += f"- {event[1]} ({event[2]}): {event[3]} at {event[4]}\n" 
             
            response = openai.ChatCompletion.create( 
                model="gpt-4", 
                messages=[ 
                    {"role": "system", "content": """You are a helpful event recommender. Consider the weather conditions  
                    and suggest suitable events. For outdoor events, consider the temperature and weather conditions. 
                    Be specific about why you recommend certain events over others. Keep your response concise but informative. 
                    If weather data is unavailable, focus on providing a balanced recommendation of both indoor and outdoor events."""}, 
                    {"role": "user", "content": context} 
                ] 
            ) 
            return response.choices[0].message.content 
        except Exception as e: 
            raise Exception(f"Recommendation error: {str(e)}") 
 
class CoordinatorAgent: 
    def __init__(self, weather_api_key, openai_api_key): 
        self.weather_agent = WeatherAgent(weather_api_key) 
        self.event_agent = EventAgent() 
        self.recommendation_agent = RecommendationAgent(openai_api_key) 
     
    def get_recommendations(self, location, date): 
        try: 
            # Get weather data 
            print(f"\nFetching weather data for {location} on {date}...") 
            weather_data = self.weather_agent.get_weather(location, date) 
             
            # Get events 
            print("Fetching events...") 
            events = self.event_agent.get_events(date) 
             
            if not events: 
                return "No events found for this date." 
             
            # Generate recommendations 
            print("Generating recommendations...") 
            recommendations = self.recommendation_agent.generate_recommendation( 
                weather_data, events 
            ) 
             
            return recommendations 
             
        except Exception as e: 
            return f"Error: {str(e)}" 
 
if __name__ == "__main__": 
    # Test dates 
    test_cases = [ 
        ("2025-07-15", "Singapore"), 
        ("2025-07-16", "Singapore"), 
        ("2025-07-17", "Singapore"),  # No events on this date 
    ] 
     
    coordinator = CoordinatorAgent(WEATHER_API_KEY, OPENAI_API_KEY) 
     
    for test_date, test_location in test_cases: 
        print(f"\n{'='*50}") 
        print(f"Getting recommendations for {test_location} on {test_date}:") 
        print(f"{'='*50}") 
        print(coordinator.get_recommendations(test_location, test_date)) 

```

**Example output:** 

```
================================================== 
Getting recommendations for Singapore on 2025-07-15: 
================================================== 

Fetching weather data for Singapore on 2025-07-15... 
Fetching events... 
Generating recommendations... 
Given the hot temperature of 31.0°C and partly cloudy weather, I recommend the indoor Art Exhibition at City Gallery. This would be a more comfortable option as it provides shelter from the potential heat, while allowing you to enjoy some inspiring modern art. However, if you enjoy warm weather, the Summer Concert at Central Park could also be an option. Just ensure to stay hydrated and protected from the sun. 

================================================== 
Getting recommendations for Singapore on 2025-07-16: 
================================================== 

Fetching weather data for Singapore on 2025-07-16... 
Fetching events... 
Generating recommendations... 
Given the warmer temperature of 31.0°C, I recommend the Theater Show at the Grand Theater. This indoor event allows you to enjoy the experience without worrying about the heat. However, if you don't mind the warmer weather, the Food Festival at the waterfront could be an exciting event where you can explore international cuisines. It's just advisable to wear a hat and sun protection if you choose the outdoor event. 

================================================== 
Getting recommendations for Singapore on 2025-07-17: 
================================================== 

Fetching weather data for Singapore on 2025-07-17... 
Fetching events... 
No events found for this date. 
```


**Testing** 

Basic Functionality: 

- Weather integration works with current conditions 
- Event retrieval from database successful 
- Recommendations consider both weather and event type 
- System handles dates with no events 

Error Cases: 

- Invalid locations will raise Weather API error 
- Dates without events return appropriate message 
- Database errors are properly caught and reported 

**Extension Activities** 

Add more sophisticated event filtering: 

- Filter by time of day 
- Add price ranges to events 
- Include event capacity/availability 
- Enhance recommendations: 
- Add transport suggestions based on weather 
- Include alternative dates for outdoor events 
- Group recommendations by time of day 
- Improve weather handling: 
- Add precipitation probability 
- Include UV index for outdoor events 
- Consider historical weather patterns 

Remember: Keep your API keys secure and never commit them to version control! 

**Submission Requirements** 

Create a document containing: 

- Working code 
- Sample outputs 
- Error handling examples 
- Any improvements implemented 

**Common Issues** 

API Rate Limits: 

- Weather API quotas 
- OpenAI rate limits 
- Data Synchronization: 
- Weather forecast availability 
- Event date matching 
- Response Quality: 
- Recommendation relevance 
- Weather condition interpretation 

**Remember**: 

- Keep API keys secure 
- Test with various weather conditions 
- Validate event data integrity 
- Handle all error cases gracefully 

**Please note** 

Note: The field of AI is evolving rapidly. By the time you work on these exercises: 

- APIs and services mentioned may have changed or been discontinued 
- Free tiers might no longer be available 
- Pricing structures could be different 
- Python libraries may have new versions with different syntax 
- Example code might need adaptation 

The core concepts remain valid, but you may need to: 

- Find alternative services 
- Adapt code to current API versions 
- Use different model versions 
- Modify prompts for newer models 
- Research current best practices 

This is normal in AI development. Learning to adapt to these changes is part of working with AI tools. 

**Resources:** 

- Check current documentation 
- Review service status pages 
- Join developer communities 
- Research alternatives 


**Estimated time:** 60-90 minutes

**Submission Instructions:**

- Select the **Start Assignment** button at the top right of this page.
- Upload your answers in the form of a Word or PDF file.
- Select the **Submit Assignment** button to submit your responses.

*This is a graded and counts towards programme completion. You may attempt this assignment only once.*


## Import Libraries and Load Environment


In [1]:
import requests
import json
from dotenv import load_dotenv
import os
from datetime import datetime
import sqlite3

# LangChain imports
from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent, initialize_agent, AgentType
from langchain.tools import tool # for @tool
# from langchain_core.tools import Tool # Traditional Tool class
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate

# Transformers for Qwen
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

# OpenAI Models: from langchain_openai import ChatOpenAI
# Gemini Models: from langchain_google_palm import ChatGooglePalm
# Ollama Models: from langchain_ollama import ChatOllama


C:\Users\Alkaff\anaconda3\envs\lc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

## 📌 Part 1: Create Event Database

In [3]:
# !!! Setup Database - RUN ONLY ONCE !!!
# --------------------------------------

def setup_database(): 
    conn = sqlite3.connect('events.db') 
    c = conn.cursor() 
     
    c.execute(''' 
        CREATE TABLE IF NOT EXISTS events ( 
            id INTEGER PRIMARY KEY, 
            name TEXT, 
            type TEXT,  -- 'indoor' or 'outdoor' 
            description TEXT, 
            location TEXT, 
            date TEXT 
        ) 
    ''') 
     
    # Sample events 
    events = [ 
        ('Summer Concert', 'outdoor', 'Live music in the park', 'Central Park', '2025-07-15'), 
        ('Art Exhibition', 'indoor', 'Modern art showcase', 'City Gallery', '2025-07-15'), 
        ('Food Festival', 'outdoor', 'International cuisine', 'Waterfront', '2025-07-16'), 
        ('Theater Show', 'indoor', 'Classical drama', 'Grand Theater', '2025-07-16') 
    ] 
     
    c.executemany('INSERT OR IGNORE INTO events (name, type, description, location, date) VALUES (?,?,?,?,?)', events) 
    conn.commit() 
    conn.close() 
 

In [4]:
# !!! Setup Database - RUN ONLY ONCE !!!
# --------------------------------------
setup_database()


In [5]:
# Testing DB
# ----------

# Verify the setup 
conn = sqlite3.connect('events.db') 
c = conn.cursor() 

print("Events table:") 
c.execute("SELECT * FROM events") 
print(c.fetchall()) 

conn.close()

Events table:
[(1, 'Summer Concert', 'outdoor', 'Live music in the park', 'Central Park', '2025-07-15'), (2, 'Art Exhibition', 'indoor', 'Modern art showcase', 'City Gallery', '2025-07-15'), (3, 'Food Festival', 'outdoor', 'International cuisine', 'Waterfront', '2025-07-16'), (4, 'Theater Show', 'indoor', 'Classical drama', 'Grand Theater', '2025-07-16')]


## 📌 Part 2: Create Weather Agent

- We will use __http://api.weatherapi.com/v1/forecast.json__ endpoint for the specific forecast
- The agent simply sends a GET request to the endpoint with the parameters
- The `get_weather()` method returns the entire json
- The `get_weather_as_string()` returns a string suitable to be used as context by the Recommendation Agent


In [6]:
# Weather Agent
# -------------

class WeatherAgent:
    def __init__(self):
        """
        Initialize with the Weather API key.
        """
        self.api_key = os.getenv("WEATHER_API")

    def get_weather(self, location, date):
        """
        Fetch current weather data for the specified location.
        Ignores date since we only get current weather.
        """
        url = "http://api.weatherapi.com/v1/forecast.json"
        params = {
            "key": self.api_key,
            "q": location,
            "aqi": "no",
            "dt": date,
            "days": 1,
            "hour": 12
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Raises HTTPError if response not 200
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Weather API error: {str(e)}")

    def get_weather_as_string(self, location, date):
        data_string = "Weather data not available."
        
        try:
            raw_data = self.get_weather(location, date)
            cond = raw_data["forecast"]["forecastday"][0]["day"]["condition"]["text"]
            t1 = raw_data["forecast"]["forecastday"][0]["day"]["maxtemp_c"]
            t2 = raw_data["forecast"]["forecastday"][0]["day"]["mintemp_c"]
            
            data_string = f"Weather Condition: {cond}, Maximum Temperature: {t1}°C, Minimum Temperature: {t2}°C."
            
        except requests.exceptions.RequestException as e:
            data_string = "Weather data not available."
        except Exception as e:
            data_string = "Weather data not available."
        return data_string


In [7]:
# Testing Sample
# --------------

weather_obj = WeatherAgent()
data = weather_obj.get_weather("Singapore","2025-07-15")
print(weather_obj.get_weather_as_string("Singapore","2025-07-15"))
print(data["forecast"])
print()
print(data)

Weather Condition: Patchy rain nearby, Maximum Temperature: 29.3°C, Minimum Temperature: 27.4°C.
{'forecastday': [{'date': '2025-07-15', 'date_epoch': 1752537600, 'day': {'maxtemp_c': 29.3, 'maxtemp_f': 84.7, 'mintemp_c': 27.4, 'mintemp_f': 81.3, 'avgtemp_c': 28.4, 'avgtemp_f': 83.1, 'maxwind_mph': 11.6, 'maxwind_kph': 18.7, 'totalprecip_mm': 1.01, 'totalprecip_in': 0.04, 'totalsnow_cm': 0.0, 'avgvis_km': 9.9, 'avgvis_miles': 6.0, 'avghumidity': 75, 'daily_will_it_rain': 1, 'daily_chance_of_rain': 82, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'uv': 6.0}, 'astro': {'sunrise': '07:05 AM', 'sunset': '07:16 PM', 'moonrise': '11:02 PM', 'moonset': '10:36 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 82, 'is_moon_up': 1, 'is_sun_up': 1}, 'hour': [{'time_epoch': 1752552000, 'time': '2025-07-15 12:00', 'temp_c': 28.6, 'temp_f': 83.5, 'is_day': 1, 'condition

## 📌 Part 3: Create Event Agent

- The agent simply queries from the database according to the parameters
- The `get_events()` method returns all the events for the date
- The `get_events_as_string()` returns a string suitable to be used as context by the Recommendation Agent


In [8]:
# Event Agent
# -----------

class EventAgent:
    def __init__(self, db_path="events.db"):
        self.db_path = db_path

    def get_events(self, date, event_type=None):
        conn = sqlite3.connect(self.db_path)
        c = conn.cursor()
        try:
            if event_type:
                c.execute(
                    'SELECT * FROM events WHERE date = ? AND type = ?',
                    (date, event_type)
                )
            else:
                c.execute(
                    'SELECT * FROM events WHERE date = ?',
                    (date,)
                )
            events = c.fetchall()
            return events
        except sqlite3.Error as e:
            raise Exception(f"Database error: {str(e)}")
        finally:
            conn.close()

    def get_events_as_string(self, date, event_type=None):
        try:
            events = self.get_events(date, event_type)
            if not events:
                return "No events found."
            
            result = "Events available:\n"
            for ev in events:
                result += (
                    f"- {ev[1]} ({ev[2]}): {ev[3]} at {ev[4]}\n"
                )
            return result.strip()
        except Exception as e:
            return "Event data not available."


In [9]:
# Testing Sample
# --------------

event_obj = EventAgent()
print(event_obj.get_events("2025-07-15"))
print(event_obj.get_events("2025-07-15","indoor"))
print()
print(event_obj.get_events_as_string("2025-07-15"))
print(event_obj.get_events_as_string("2025-07-15","indoor"))

[(1, 'Summer Concert', 'outdoor', 'Live music in the park', 'Central Park', '2025-07-15'), (2, 'Art Exhibition', 'indoor', 'Modern art showcase', 'City Gallery', '2025-07-15')]
[(2, 'Art Exhibition', 'indoor', 'Modern art showcase', 'City Gallery', '2025-07-15')]

Events available:
- Summer Concert (outdoor): Live music in the park at Central Park
- Art Exhibition (indoor): Modern art showcase at City Gallery
Events available:
- Art Exhibition (indoor): Modern art showcase at City Gallery


## 📌 Part 4: Recommendation Agent

- The agent uses an LLM to make use of the weather and event data to recommend the event according to the parameters
- The LLM will be passed in as an argument in the constructor
- The `generate_recommendation()` method returns the suggestion in natural language


In [10]:
# Recommendation Agent
# --------------------

class RecommendationAgent:
    def __init__(self, llm):
        """
        Initialize with the LLM instance (e.g., Qwen model wrapped with LangChain).
        """
        self.llm = llm

    def generate_recommendation(self, weather_text, events_text):
        """
        Combine weather and events information, pass to LLM to get a natural language recommendation.
        """
        # Create the context prompt
        prompt = f"""
You are a helpful event recommender.

Here is today's weather forecast:
{weather_text}

Here are today's events:
{events_text}

Based on the weather conditions and available events, suggest the most suitable activities.
If the weather includes rain, only recommend indoor events.
Be specific about why you recommend certain events over others.
Keep your response to ONE sentence only.
If weather data is unavailable, still recommend any available events.
If the events data is unavailable, say "No events to recommend."
IMPORTANT: Start your answer with "Answer:" and nothing else.
"""
        # Call the LLM
        response = self.llm.invoke(prompt)
        return response


In [11]:
# Load LLM - Qwen 1.5 1.8B Chat
# -----------------------------

# Load Model
model_id = "Qwen/Qwen1.5-1.8B-Chat"
tok  = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            device_map="auto",          # "cuda" if GPU, else "cpu"
            torch_dtype="auto"
        )

# Create Qwen Pipeline
qwen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    #max_new_tokens=512,
    temperature=0.2,
    #device=0  # device=0 for GPU
)

# Create the LLM
llm = HuggingFacePipeline(pipeline=qwen_pipeline)


Device set to use cpu
C:\Users\Alkaff\AppData\Local\Temp\ipykernel_8056\3132568482.py:25: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=qwen_pipeline)


In [12]:
# Testing Sample
# --------------

recc_obj = RecommendationAgent(llm)
weather_text = weather_obj.get_weather_as_string("Singapore","2025-07-15")
event_text = event_obj.get_events_as_string("2025-07-15")
print(recc_obj.generate_recommendation(weather_text,event_text))



You are a helpful event recommender.

Here is today's weather forecast:
Weather Condition: Patchy rain nearby, Maximum Temperature: 29.3°C, Minimum Temperature: 27.4°C.

Here are today's events:
Events available:
- Summer Concert (outdoor): Live music in the park at Central Park
- Art Exhibition (indoor): Modern art showcase at City Gallery

Based on the weather conditions and available events, suggest the most suitable activities.
If the weather includes rain, only recommend indoor events.
Be specific about why you recommend certain events over others.
Keep your response to ONE sentence only.
If weather data is unavailable, still recommend any available events.
If the events data is unavailable, say "No events to recommend."
IMPORTANT: Start your answer with "Answer:" and nothing else.
Answer:

"Considering the patchy rain nearby and the maximum temperature of 29.3°C, I would recommend attending the Summer Concert in Central Park as it offers outdoor live music and is likely to be he

In [13]:
# Testing Sample
# --------------

recc_obj = RecommendationAgent(llm)
weather_text = weather_obj.get_weather_as_string("Singapore","2025-07-16")
event_text = event_obj.get_events_as_string("2025-07-16")
print(recc_obj.generate_recommendation(weather_text,event_text))



You are a helpful event recommender.

Here is today's weather forecast:
Weather Condition: Patchy rain nearby, Maximum Temperature: 29.2°C, Minimum Temperature: 28.0°C.

Here are today's events:
Events available:
- Food Festival (outdoor): International cuisine at Waterfront
- Theater Show (indoor): Classical drama at Grand Theater

Based on the weather conditions and available events, suggest the most suitable activities.
If the weather includes rain, only recommend indoor events.
Be specific about why you recommend certain events over others.
Keep your response to ONE sentence only.
If weather data is unavailable, still recommend any available events.
If the events data is unavailable, say "No events to recommend."
IMPORTANT: Start your answer with "Answer:" and nothing else.
Answer:

Answer: Based on the patchy rain nearby, I would recommend attending the Theater Show at Grand Theater as it offers a classical drama performance indoors and is likely to be less affected by the rain. 

## 📌 Part 5: Coordinator Agent

- The agent coordinates the 3 other agents (Weather, Events, Recommendation) to get the final output
- The `get_recommendations()` method returns the final output


In [14]:
# Coordinator Agent
# -----------------

class CoordinatorAgent:
    def __init__(self, weather_agent, event_agent, recommendation_agent):
        """
        Initializes the coordinator with the three agents.
        """
        self.weather_agent = weather_agent
        self.event_agent = event_agent
        self.recommendation_agent = recommendation_agent

    def get_recommendations(self, location, date):
        """
        Orchestrates weather lookup, event lookup, and recommendation generation.
        Returns the final recommendation text.
        """
        try:
            # Get weather string
            print("Fetching weather...")
            weather_text = self.weather_agent.get_weather_as_string(location, date)

            # Get event string
            print("Fetching events...")
            events_text = self.event_agent.get_events_as_string(date)

            # Generate recommendation
            print("Generating recommendation...")
            recommendation = self.recommendation_agent.generate_recommendation(
                weather_text, events_text
            )
            recommendation = recommendation.split("Answer:")[-1].strip()
            return recommendation

        except Exception as e:
            return f"Error: {e}"


In [15]:
# Testing Sample
# --------------

# Initialize agents
weather_agent = WeatherAgent()
event_agent = EventAgent()
recommendation_agent = RecommendationAgent(llm)

# Initialize coordinator
coordinator = CoordinatorAgent(weather_agent, event_agent, recommendation_agent)

# Test 
print(coordinator.get_recommendations("Singapore", "2025-07-16"))


Fetching weather...
Fetching events...
Generating recommendation...
Attend the Food Festival at Waterfront as it offers international cuisine in an outdoor setting amidst patchy rain."


## 📌 Part 6: Testing Everything

- Initialize the LLM again
- Create all the objects again
- Run through the test cases


In [16]:
# Load LLM - Qwen 1.5 1.8B Chat
# -----------------------------

# Load Model
model_id = "Qwen/Qwen1.5-1.8B-Chat"
tok  = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            device_map="auto",          # "cuda" if GPU, else "cpu"
            torch_dtype="auto"
        )

# Create Qwen Pipeline
qwen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    #max_new_tokens=512,
    temperature=0.2,
    #device=0  # device=0 for GPU
)

# Create the LLM
llm = HuggingFacePipeline(pipeline=qwen_pipeline)


Device set to use cpu


In [17]:
# Initialize Objects for Testing
# ------------------------------

# Initialize agents
weather_agent = WeatherAgent()
event_agent = EventAgent()
recommendation_agent = RecommendationAgent(llm)

# Initialize coordinator
coordinator = CoordinatorAgent(weather_agent, event_agent, recommendation_agent)

# Test queries
test_cases = [
    ("Singapore", "2025-07-15"),
    ("Singapore", "2025-07-16"),
    ("Singapore", "2025-07-17"),  # No events
]


In [18]:
# !!! TEST !!!
# ------------

for loc, dt in test_cases:
    print("="*50)
    print(f"Recommendations for {loc} on {dt}")
    print("="*50)
    print(coordinator.get_recommendations(loc, dt))
    print()


Recommendations for Singapore on 2025-07-15
Fetching weather...
Fetching events...
Generating recommendation...
"Considering the patchy rain nearby and maximum temperature of 29.3°C, I would recommend attending the Summer Concert in Central Park as it offers outdoor live music in a beautiful outdoor setting amidst lush greenery. The art exhibition at City Gallery, on the other hand, would be a great option for those interested in modern art, as it features an indoor showcase showcasing contemporary works."

Recommendations for Singapore on 2025-07-16
Fetching weather...
Fetching events...
Generating recommendation...
Based on the patchy rain nearby, I would recommend attending the Food Festival at Waterfront as it offers international cuisine and an outdoor setting for a fun and enjoyable experience under the warm weather. The theater show at Grand Theater, on the other hand, could be a great option if the rain doesn't last long or if the indoor venue is accessible. If the weather data

## 🏁 Conclusion

For this assignment, I decided to **challenge myself** by using an **open-source model (Qwen 1.5 Chat)** instead of the recommended OpenAI GPT-4. Key observations and adaptations:

The solution was implemented using **three modular agents**:

- `WeatherAgent` for fetching weather forecasts via API.
- `EventAgent` for retrieving local events from an SQLite database.
- `RecommendationAgent` for generating final recommendations using the LLM.

The final `CoordinatorAgent` class orchestrates data retrieval and combines the results into a single output.

**Qwen 1.5b Chat demonstrated inconsistent instruction adherence:**

- Sometimes it generated multi-paragraph recommendations even when asked for a single sentence.
- It occasionally recommended outdoor events despite rain, showing limited reasoning alignment.
- The model tended to "hallucinate" fallback suggestions (e.g., cooking classes) even though the prompt disallowed it.
- These issues are likely because **Qwen is not strongly instruction-tuned** compared to models like GPT-4 Turbo.

Prompting required extra experimentation to achieve acceptable outputs:

- Repeatedly rephrasing instructions to emphasize brevity and relevance.
- Lowering generation temperature to reduce verbosity.

Despite these challenges, the solution achieved the core objectives:

- It combined **multi-agent coordination** (weather + events).
- It generated **context-aware recommendations**.
- It demonstrated end-to-end autonomous reasoning, query execution, and result formatting.

This assignment illustrates that **open-source models can be adapted for multi-agent systems**, but additional prompt engineering effort is necessary to ensure reliability.
